# Машинный перевод и озвучивание видеозаписей на русском языке

## О проекте

[Есть](https://www.youtube.com/watch?v=p3lsYlod5OU&ab_channel=LexFridman) интересная беседа [Лекса Фридмана](https://en.wikipedia.org/wiki/Lex_Fridman) с [Михаилом Левиным](https://en.wikipedia.org/wiki/Michael_Levin_(biologist)). Несмотря, на русское происхождение обоих собеседников разговор они ведут на английском. Цель проекта перевести и озвучить беседу на русском языке.

В проекте используется [готовая расшифровка с таймингом](https://karpathy.ai/lexicap/0325-large.html), созданная с помощью пакета [OpenAI Wisper](https://github.com/openai/whisper).
Далее описывается парсинг расшифровки, перевод на русский язык, машинное озвучивание, выравнивание длины русских фрагментов с соответствующими оригинальными.

Данный подход можно использовать для перевода и озвучивания любой видеозаписи.

## Устанавливаем библиотеки

In [ ]:
# %pip install tqdm
# %pip install python-docx
# %pip install -q torchaudio omegaconf

## Импортируем необходимые для работы модули

In [ ]:
from tqdm import tqdm
from docx import Document
from bs4 import BeautifulSoup
from datetime import datetime
from scipy.io.wavfile import read, write

import os
import torch
import requests
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
from IPython.display import Audio, display

torch.hub.download_url_to_file('https://raw.githubusercontent.com/snakers4/silero-models/master/models.yml',
                               'latest_silero_models.yml',
                               progress=False)

## Парсинг расшифровки с таймингом

Для парсинга используем библиотеку `BeatifulSoup`:

In [ ]:
url = 'https://karpathy.ai/lexicap/0325-large.html'
data = requests.get(url).text
soup = BeautifulSoup(data)

## Обработка английского текста

Выделим фрагменты текста и их тайминги в отдельные списки:

In [ ]:
t_divs = soup.find_all('div', {'class': 't'})
en_text = []
for div in t_divs:
    en_text.append(div.text)

s_divs = soup.find_all('div', {'class': 's'})
timing = []
for div in s_divs:
    timing.append(div.a.text)

Важно в процессе работы над текстом не потерять ни одного символа.

In [ ]:
text_size = 0
for row in en_text:
    text_size += len(row)

text_size

Почти две сотни знаков.

Для удовлетворительного качества необходимо переводить текст как минимум целыми предложениями. В качестве знаков для разделения используем точку и вопросительный знак.

In [ ]:
def sign_finder(sign, str):
    '''Finds the last position of the sing in the string'''
    rev_pos = str[::-1].find(sign)
    if rev_pos == -1:
        return 0    
    pos = len(str) - rev_pos
    return pos
def pos_finder(text):
    signs = ['.', '?']
    pos = []
    for sign in signs:
        pos.append(sign_finder(sign, text))
    return max(pos)

Перенесём незаконченные фрагменты предложений к фрагментам, содержащим знаки конца предложения, и внесём соответствующие поправки в тайминг.

In [ ]:
dense_text = []
new_timing = []
buffer = []
time = []
for i, text in enumerate(en_text):
    pos = pos_finder(text)
    if (pos == 0) & (i < len(en_text) - 1):
        buffer.append(text)
        time.append(timing[i])
    elif len(buffer) > 0:
        merged = ""
        for buf_item in buffer:
            merged += buf_item 
        dense_text.append(merged + text)
        new_timing.append(time[0])
        buffer.clear()
        time.clear()
    else:
        dense_text.append(text)
        new_timing.append(timing[i])


In [ ]:
dense_size = 0
for row in dense_text:
    dense_size += len(row)
dense_size == text_size

Пока всё на месте.

Из данных списков создадим таблицу `Pandas`: 

In [ ]:
dict = {'timing': new_timing, 'en_text': dense_text}
df = pd.DataFrame(dict)
print(df.en_text.iloc[40])
df

Тайминги переведем в формат `DateFrame` и вычислим продожительность оригинальных фрагментов, сохранив их в отдельный столбец:

In [ ]:
format = '%H:%M:%S.%f'
def duration_calculator(row):
    '''Calculates duration speech fragment in a row'''
    idx = row.name
    time1 = row['timing'] + '000'
    if idx == len(df) - 1:
        return datetime.strptime("03:00:20.000000", format) - datetime.strptime(time1, format)    
    time2 = df.loc[idx + 1, 'timing'] + '000'
    return datetime.strptime(time2, format) - datetime.strptime(time1, format)

df['duration'] = df.apply(duration_calculator, axis=1)
df['duration'] = df['duration'].apply(lambda x: x.total_seconds())
df

Посмотрим на длину фрагментов:

In [ ]:
df.en_text.apply(len).describe()

Самый короткий 5 символов, а длинный -- 574. 

Для перевода фрагментов текста в отдельные предложения (группы предложений) и определения соответствующей им временной продолжительности мне будет удобнее работать со списками, а не таблицей.

In [ ]:
text = list(df.en_text)
duration = list(df.duration)

Будем ориентироваться на знаки окончания предложения в конце фрагмента:

In [ ]:
def min_pos_finder(text):
    '''Returns the first position of the signs in the text'''
    signs = ['.', '?']
    pos = []
    for sign in signs:
        pos.append(text.find(sign))
    if min(pos) == -1:
        return max(pos) + 1
    else:
        return min(pos) + 1

Собственно переносим, что нужно в тексте и вносим, соответствующие изменения в их длительность:

In [ ]:
for i, row in enumerate (text):
    if i != 0:
        pos = min_pos_finder(row)
        head = row[:pos]
        tail = row[pos:]
        duration[i - 1] = duration[i - 1] + duration[i] * len(head) / len(row)
        duration[i] = duration[i] * len(tail) / len(row)
        text[i - 1] = text[i - 1] + head
        text[i] = tail         


In [ ]:
size = 0
for row in text:
    size += len(row)

print('Количество знаков в исходном тексте: ', df.en_text.apply(len).sum())
print('Количество знаков итоговом тексте: ', size)
print('Исходная общая продолжительность: ', sum(duration))
print('Общая продолжительность после обработки: ', df.duration.sum())


Всё на месте. Возвращаем данные обратно в таблицу.

In [ ]:
df['en_text'] = text
df['duration'] = duration

In [ ]:
df

In [ ]:
len(df)

## Перевод текста на русский

Для перевода будем использовать десктопный вариант переводчика DeepL. Из-за ограничения в 100 000 символов нам потребуется несколько текстовых документов.

In [ ]:
lists = []
for i in range(len(text)):
    lists.append([text[i], duration[i]])

In [ ]:
batch_size = 500
start = 0
for i in range(round(len(text) / batch_size)):
    document = Document()
    table = document.add_table(rows=0, cols=3)

    batch = lists[start:start+batch_size]

    for j, row in enumerate(batch):
        row_cells = table.add_row().cells
        row_cells[0].text = f'{j}'
        row_cells[1].text = row[0]
        row_cells[2].text = f'{row[1]:.4f}'

    start += batch_size
    document.save(f'tab_{i + 1}.docx')



In [ ]:
ru_text = []
for i in range(round(len(text) / batch_size)):
    document = Document(f'tab_{i + 1} ru.docx')
    table = document.tables[0]
    for cell in table.column_cells(1):
        ru_text.append(cell.text)

In [ ]:
document = Document()
for txt in ru_text:
    document.add_paragraph(txt)
document.save('ru_text.docx')

In [ ]:
ru_text = []
document = Document('ru_text.docx')
for p in document.paragraphs:
    ru_text.append(p.text[1:])

In [ ]:
len(ru_text)

## Машинное озвучивание текста и подготовка к монтажу

В качестве синтеза речи будем использовать замечательные модели от [Silero](https://colab.research.google.com/github/snakers4/silero-models/blob/master/examples_tts.ipynb#scrollTo=_-S9KQ19mzpy), которые доступны всем желающим для некомерческого использования:

In [ ]:
models = OmegaConf.load('latest_silero_models.yml')
language = 'ru'
model_id = 'v3_1_ru'
device = torch.device('cpu')
model, example_text = torch.hub.load(repo_or_dir='snakers4/silero-models',
                                     model='silero_tts',
                                     language=language,
                                     speaker=model_id)
model.to(device)  # gpu or cpu

Пустых элементов быть не должно.

In [ ]:
# ru_text[377] = 'Это определенно стимулирует взгляд на себя и внешний мир как на объект, я думаю, это неизбежно.'

In [ ]:
def write_wave(path, audio, sample_rate):
    """Writes a .wav file.
    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)
for i, txt in tqdm(enumerate(ru_text)):
  txt = f'<speak><prosody rate="fast">{txt}</prosody></speak>'
  sample_rate = 48000
  speaker = 'xenia'
  audio = model.apply_tts(ssml_text=txt,
                        speaker=speaker,
                        sample_rate=sample_rate)
  write_wave(path=f'files/{i}-1.wav', audio=(audio * 32767).numpy().astype('int16'), sample_rate=48000)

In [ ]:
# df = pd.DataFrame({'duration': duration, 'en_text': text, 'ru_text': ru_text})
# df.to_csv('levin.csv', index=False)

Теперь, даже после перерыва, мы можем загрузить таблицу из файла:

In [ ]:
df = pd.read_csv('levin.csv')

Создадим набор звуковых файлов из текстовых фрагментов:

## Обработка звука

Сравним длину полученных фрагментов с длительностью оригинальных фрагментов:

In [ ]:
durs = []
for i in range(len(df)):
    file_name = 'files/' + f'{i}-1.wav'
    fs, data = read(file_name)
    durs.append(round(len(data) / fs, 3))   
sum(durs)

Общая длительность озвучивания 9555 секунд,  меньше общей длительности видеозаписи -- 10820 секунд.

Найдём разность между оригинальной длительностью фрагмента и русской озвучкой:

In [ ]:
df['ru_duration'] = durs
df['delta_duration'] = df['duration'] - df['ru_duration']

In [ ]:
df

In [ ]:
df.delta_duration.describe()

Оригинальный фрагмент может быть на 14 секунд длиннее, а может и почти на 6 секунд короче.

In [ ]:
df.delta_duration.sum() + df.ru_duration.sum()

Общая продолжительность совпадает с оригинальной.

Если длительности озвученных фрагментов превышает оригинал, то заберём это время у следующих фрагментов, обладающих запасом по времени. Работать будем со списком. 

In [ ]:
deltas = list(df.delta_duration)

In [ ]:
time = 0
for i, delta in enumerate(deltas):
    
    if delta < 0:        
        time += delta
        deltas[i] = 0
    elif time < 0:
        if abs(time) <= delta:
            deltas[i] += time
            time = 0
        else:
            time += delta 
            deltas[i] = 0       

In [ ]:
print(df.duration.sum())
print(df.ru_duration.sum() + df.delta_duration.sum())

Общее время не поменялось.

Добавим информацию в таблицу.

In [ ]:
df['delta_duration'] = deltas

In [ ]:
df

In [ ]:
fs = 48000
for i, delta in enumerate(deltas):
  if delta > 0:
    multiplicator = int(delta * fs)
    spacer = np.array([0] * multiplicator).astype('int16')
    write_wave(
      path=f'{i}.wav',
      audio=spacer,
      sample_rate=fs)

In [ ]:
d_durs = []
for i, delta in enumerate(deltas):
  if delta > 0:
    file_name = f'files/{i}-1.wav'
    fs, data = read(file_name)
    d_durs.append(round(len(data) / fs, 3))   
sum(d_durs)

In [ ]:
df.delta_duration.sum()

Добавим после каждого вновь озвученного звукового фрагмента, продолжительностью меньше оригинального, заполненной тишиной спейсер, необходимой продолжительности:

Можно попробовать "сшить" фрагменты русской озвучки со спейсерами с помощью утилиты [`ffmpeg`](https://ffmpeg.org/download.html). 

In [ ]:
ru_text[377]

In [ ]:
with open("list.txt", "w") as f:
    for i in range(len(df)):
        filename = f'files/{i}.wav'
        line = f"file '{filename}'\n"
        if deltas[i] > 0:
            line += f"file 'files/{i}-1.wav'\n"
        f.write(line)

In [ ]:
os.system("ffmpeg -f concat -i out/list.txt -c copy out/output.wav")

По неизвестной мне причине, итоговый файл у меня получился размером больше, чем должен был. Я использовал для объединения утилиту плеера `AIMP`, указав папку `files` как источник и один файл с раширением `.mp3`. Последний вариант хорошо сработал и я получил на выходе звуковой файл продолжительностью 3 часа 20 секунд (10820 секунд).

Далее я загружаю оригинальный файл с видео на ПК с помощью сервиса [SaveFrom](https://ru.savefrom.net/1-%D0%B1%D1%8B%D1%81%D1%82%D1%80%D1%8B%D0%B9-%D1%81%D0%BF%D0%BE%D1%81%D0%BE%D0%B1-%D1%81%D0%BA%D0%B0%D1%87%D0%B0%D1%82%D1%8C-%D1%81-youtube-130/?url=http%3A%2F%2Fyoutube.com%2Fwatch%3Fv%3Dp3lsYlod5OU&ab_channel=LexFridman&utm_source=youtube.com&utm_medium=short_domains&utm_campaign=ssyoutube.com&a_ts=1666604287.019) и в видеоредакторе приглушаю оригинальную видеодорожку и добавляю вновь созданную. Можно попробовать обойтись средствами утилиты `ffmpeg`.


## Общий вывод

Результат можно посмотреть [здесь](https://youtu.be/7HZnJYKBXNo)

На мой взгляд результат получился значительно лучше чем в предыдущей итерации. Качество перевода улучшилось, хотя и далеко не идеальное, в этом направлении следует продолжить работать. Качество озучивание значительно улучшилось, однако остались проблемы с ударениями, буквы ё пришлось выделять самостоятельно.

Крайне желательно использовать надежный нормализатор текста. Входящий в состав моделей Silero (если он там вообще есть) работает неудовлетворильно, мне пришлось делать много работы самостоятельно.
Можно попробовать реализовать на основе данного проекта суфлёр и озвучить текст перевода самостоятельно.